In [24]:
import random
import pandas as pd
from together import Together
from faker import Faker
import os
import time

In [25]:
os.environ["TOGETHER_API_KEY"] = "e89932e7d4565879c28cfbfc20d59ed9811609ea306e136f85078e743e62ea5a"
client = Together()

In [26]:
roles = {
    "Data Scientist":['Python', 'Data Manipulation', 'Machine Learning', 'Statistics and Probability', 'NLP', 'Deep Learning', 'Data Visualization'],
    "Data Engineer":['Big Data Frameworks', 'Data Pipelines', 'ETL Tools', 'Databases', 'Cloud Platforms', 'Python', 'Data Warehousing', 'CI/CD Tools'],
    "Software Engineer": ['Java', 'Algorithm and Data Structure', 'Web Development', 'Backend Development', 'Version Control', 'DevOps','Testing', 'API'],
    "Product Manager": ['Strategic Planning', 'Roadmap Development','Stakeholder management','Agile and Scrum Methodologies', 'Prototyping', 'Business Communication'],
    "UI Engineer": ['Frontend Frameworks', 'Responsive Design', 'Design Tools', 'Version Control', 'Animation Libraries']
}

In [27]:
experience_levels = ["Entry-level", "Mid-level", "Senior-level", "Lead", "Director"]
work_environments = ["Remote", "Hybrid", "In-office", "On-Site"]

In [28]:
select_reasons = [
    "Provided innovative and creative solutions during the interview",
    "Displayed excellent collaboration and teamwork mindset",
    "Showcased the ability to quickly adapt and learn new concepts",
    "Demonstrated a strong understanding of the company’s goals and how they could contribute",
    "Presented clean, efficient, and well-documented code solutions"
]
reject_reasons = [
    "Failed to address performance and scalability concerns in solutions",
    "Had difficulty applying theoretical knowledge to practical scenarios",
    "Showed lack of preparedness for typical interview challenges",
    "Displayed rigidity in adapting to alternative approaches or feedback",
    "Did not demonstrate curiosity or initiative in exploring the problem space"
]

In [29]:
def get_reasons(outcome, num_reasons=2):
    if outcome == 'select':
        list = random.sample(select_reasons, k=num_reasons)
    elif outcome == 'reject':
        list = random.sample(reject_reasons, k=num_reasons)
    return ', '.join(list)

In [30]:
def call_together_api(prompt):
    response = client.chat.completions.create(
        model="meta-llama/Llama-Vision-Free",
        messages=[{"role": "user", "content": f"{prompt}"}],
    )
    content = response.choices[0].message.content
    return content.replace("**", "")

In [31]:
fake = Faker()

In [32]:
def generate_candidate_data(n):
    data = []
    for _ in range(n):
        id_formatted = f"durgba{_+1:03d}"
        name = fake.name()
        role = random.choice(list(roles.keys()))
        skillset = roles[role]
        poor_skills = random.sample(skillset, random.randint(1, min(3, len(skillset))))
        outcome = random.choice(["select","reject"])

        transcript_prompt = (
            f"Simulate an interview for a candidate named {name} applying for the role of {role}."
            f"The candidate performs poorly in the following skills: {', '.join(poor_skills)}. Generate a professional transcript."
            f"Do not generate generic statements like 'Here is a simulated interview transcript for the ...'."
        )
        transcript = call_together_api(transcript_prompt)

        resume_prompt = (
            f"Create a resume for the candidate named {name} applying for the job role of {role}. "
            f"The candidate is skilled in {', '.join([skill for skill in skillset if skill not in poor_skills])}"
            f"Do not generate generic statements like 'Here's a sample resume for the ...'."
        )
        resume = call_together_api(resume_prompt)

        expected_experience = random.choice(["0-2 years", "3-5 years", "6-8 years", "9+ years"])
        job_description = (
            f"Skilled {role} with expertise in {', '.join(random.sample(skillset,min(4,len(skillset))))}"
            f"\nExpected_experience : {expected_experience}"
        )

        data.append({
            "Id": id_formatted,
            "Name": name,
            "Role": role,
            "Transcript": transcript,
            "Resume": resume,
            "Performance": outcome,
            "Reason for decision": get_reasons(outcome),
            "Job description": job_description
        })

        time.sleep(5)

    return pd.DataFrame(data)

In [33]:
n = int(input("Enter the number of candidates: "))
data = generate_candidate_data(n)
data

,Id,Name,Role,Transcript,Resume,Performance,Reason for decision,Job description
0,durgba001,Elizabeth Smith,UI Engineer,Interview Transcript\n\nInterviewer: Rachel Le...,Elizabeth Smith\nUI Engineer\n\nContact Inform...,reject,Showed lack of preparedness for typical interv...,Skilled UI Engineer with expertise in Frontend...
1,durgba002,Amanda Campbell,Data Engineer,Interview Transcript: Data Engineer Position\n...,Amanda Campbell\nContact Information:\n\n* Ema...,reject,Had difficulty applying theoretical knowledge ...,Skilled Data Engineer with expertise in Python...
2,durgba003,Stephen Velez,Product Manager,Product Manager Interview\n\nInterviewer: Rach...,Stephen Velez\nContact Information:\n\n* Email...,reject,Displayed rigidity in adapting to alternative ...,Skilled Product Manager with expertise in Road...
3,durgba004,Guy Shaw,Data Engineer,Interview Transcript\n\nInterviewer: Rachel Pa...,Guy Shaw\nData Engineer\n\nContact Information...,select,Displayed excellent collaboration and teamwork...,Skilled Data Engineer with expertise in Python...
4,durgba005,Jeffrey Bailey,Software Engineer,Interview Transcript\n\nInterviewer: Rachel Ch...,Here is a resume for Jeffrey Bailey:\n\nJeffre...,reject,Had difficulty applying theoretical knowledge ...,Skilled Software Engineer with expertise in We...
...,...,...,...,...,...,...,...,...
195,durgba196,Pedro Lara,Data Engineer,Interview Transcript: Data Engineer Position\n...,Pedro Lara\nData Engineer\n\nContact Informati...,reject,Showed lack of preparedness for typical interv...,Skilled Data Engineer with expertise in CI/CD ...
196,durgba197,Joseph Jimenez,Data Scientist,Data Scientist Interview with Joseph Jimenez\n...,Joseph Jimenez\nData Scientist\n\nContact Info...,select,Showcased the ability to quickly adapt and lea...,Skilled Data Scientist with expertise in Data ...
197,durgba198,Scott Harvey,Software Engineer,"Interviewer: Rachel Lee, Senior Software Engin...",Scott Harvey\nContact Information:\n\n* Email:...,select,Demonstrated a strong understanding of the com...,Skilled Software Engineer with expertise in Ja...
198,durgba199,Tami Sharp,Product Manager,Interview for Product Manager Position\n\nInte...,Tami Sharp\nProduct Manager\n\nContact Informa...,select,Displayed excellent collaboration and teamwork...,Skilled Product Manager with expertise in Stak...


In [34]:
data.to_excel("candidates_list.xlsx", index=False)